In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [138]:
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [139]:
initial_train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
print(initial_train_data.shape)

print(initial_train_data.Ticket)

(891, 12)
0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object


In [140]:
# set up test data
initial_test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
print(initial_test_data.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


In [141]:
features = [
"Pclass", "Sex", "SibSp", "Parch","Ticket","Title","Embarked","FamilySize","HasCabin"
]

num_features = ["Pclass","SibSp", "Parch","HasCabin","FamilySize"]

cat_features = ["Sex","Ticket","Title","Embarked"]

In [142]:
def add_features(df):
    df = df.copy()

    df["Title"] = df["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
    df["Title"] = df["Title"].replace(
        ["Lady","Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"],
        "Rare"
    )
    df["Title"] = df["Title"].replace({"Mlle":"Miss","Ms":"Miss","Mme":"Mrs"})

    df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

    df["HasCabin"] = df["Cabin"].notnull().astype(int)

    return df



<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_55/5363721.py:5: SyntaxWarning: invalid escape sequence '\.'
  df["Title"] = df["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)


In [143]:
train_df = add_features(initial_train_data)
test_df  = add_features(initial_test_data)

train_df.dtypes

/tmp/ipykernel_55/5363721.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)
/tmp/ipykernel_55/5363721.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Title           object
FamilySize       int64
HasCabin         int64
dtype: object

In [144]:
X = train_df[features]
y = train_df['Survived']
X_test = test_df[features]

cat_feature_indices = [
    X.columns.get_loc(c) for c in cat_features
]

In [145]:
print(train_df[features].head())
print(test_df[features].head())

   Pclass     Sex  SibSp  Parch            Ticket Title Embarked  FamilySize  \
0       3    male      1      0         A/5 21171    Mr        S           2   
1       1  female      1      0          PC 17599   Mrs        C           2   
2       3  female      0      0  STON/O2. 3101282  Miss        S           1   
3       1  female      1      0            113803   Mrs        S           2   
4       3    male      0      0            373450    Mr        S           1   

   HasCabin  
0         0  
1         1  
2         0  
3         1  
4         0  
   Pclass     Sex  SibSp  Parch   Ticket Title Embarked  FamilySize  HasCabin
0       3    male      0      0   330911    Mr        Q           1         0
1       3  female      1      0   363272   Mrs        S           2         0
2       2    male      0      0   240276    Mr        Q           1         0
3       3    male      0      0   315154    Mr        S           1         0
4       3  female      1      1  3101298   Mr

In [146]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from catboost import CatBoostClassifier

In [147]:
cb = CatBoostClassifier(
    iterations=800,
    depth=7,
    learning_rate=0.03,
    l2_leaf_reg=6,
    loss_function="Logloss",
    random_seed=42,
    verbose=0,
    cat_features=cat_feature_indices
)

In [148]:
pl = Pipeline([
    ("model", cb)
])

In [149]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(
    pl,
    X,
    y,
    cv=cv,
    scoring="accuracy"
)

print("CV accuracy:", scores.mean())
print("Std:", scores.std())


CV accuracy: 0.8350134957002071
Std: 0.016156077963473302


In [150]:
pl.fit(X,y)

cb_pred = pl.predict(X_test)

In [151]:
predictions = cb_pred.astype(int)

In [152]:
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions
})

print(submission)

submission.to_csv("submission.csv", index=False)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
